Use Spark To Provide Audio Recommendations

http://www.iro.umontreal.ca/~lisa/datasets/profiledata_06-May-2005.tar.gz

million song dataset
http://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset

NameError: name 'df' is not defined